In [27]:
import pandas as pd
pd.set_option('display.max_columns', 100)
from tqdm import tqdm
tqdm.pandas()

In [290]:
vacs = pd.read_csv('/home/gk/Desktop/hh_ru_csv/hhru_vacancy.csv')
vacs = vacs[vacs['area_id']==2]
vacs = vacs[['id', 'description']]
vacs.head(3)

,id,description
26,37003152,<p>Компания «Цифровое проектирование» ведет ис...
27,37849676,"<p>Геоскан - российская группа компаний, заним..."
60,38412704,<strong>Обязанности:</strong> <ul> <li>охрана ...


In [291]:
sv = pd.read_csv('/home/gk/Desktop/hh_ru_csv/hhru_vacancy_specializations.csv')
sv = sv[sv['vacancy_id'].isin(vacs['id'])]

sv = sv.groupby('vacancy_id').agg({
    'specialization_id': lambda col: list(set(round(col, 3).tolist()))
}).reset_index(drop=False)

In [292]:
sv.head(3)

,vacancy_id,specialization_id
0,10161653,"[17.333, 17.538, 17.149, 17.183]"
1,16555352,"[1.225, 17.572, 17.324, 17.538, 17.269, 17.333]"
2,22089555,"[17.256, 29.512, 17.231]"


In [293]:
vacs = vacs.merge(sv, left_on='id', right_on='vacancy_id').drop(columns=['vacancy_id'])

In [294]:
stem_specs = [1.82, 14.87, 1.221, 18.488, 18.568, 18.57, 18.81, 18.245, 15.237, 14.87, 14.91]

In [295]:
def get_vac_spec(row):
    flag = False
    for spec in row['specialization_id']:
        if spec in stem_specs:
            flag = True
            continue
    return flag
    

vacs['flag'] = vacs.progress_apply(lambda row: get_vac_spec(row), axis=1)
vacs = vacs[vacs.flag]

100%|██████████| 24923/24923 [00:00<00:00, 70431.69it/s]


In [296]:
vacs.head(2)

,id,description,specialization_id,flag
0,37003152,<p>Компания «Цифровое проектирование» ведет ис...,"[1.221, 1.25]",True
1,37849676,"<p>Геоскан - российская группа компаний, заним...",[1.221],True


In [297]:
import string


def drop_garbage_chars(t):
    disallowed_characters =['<p>', '<strong>', '</strong>', '</p>', 'br', 
                            '</li>', '</ul>', '<li>', '<ul>', '&quot;', '/u']  \
                            + [*string.punctuation]
    disallowed_characters.remove('+')
    disallowed_characters.remove('/')

    for character in disallowed_characters:
        t = t.replace(character, "")
    
    # t = t.lower()
    
    return t


def split_text(t, idx, lst):
    if t:
        for c, v in enumerate(lst):
            try:
                t = t.split(v, 1)[idx]
                return t
            except IndexError:
                if c != len(lst)-1:
                    continue
                else:
                    return None
    else:
        return None


def get_skills(text):
    text = drop_garbage_chars(text)

    splitters_dict = {
        'requirements': ['Требования', 'Обязанности', 'ожидания'],
        'terms': ['Условия', 'условия',  'предлагаем']
        }

    idx_dict = {
        'requirements': 1,
        'terms': 0
        }
    
    for splitter in ['requirements', 'terms']:
        text = split_text(text, idx_dict[splitter], splitters_dict[splitter])     

    return text


vacs['desc2'] = vacs['description'].progress_map(lambda x: get_skills(x))

print(vacs['desc2'].dropna().shape[0])
print(vacs['desc2'].shape[0])
print('% вакансий с требованиями: ', round(vacs['desc2'].dropna().shape[0] / vacs['desc2'].shape[0], 3)*100, '%')

vacs.dropna(subset=['desc2'], inplace=True)

100%|██████████| 5731/5731 [00:01<00:00, 5697.18it/s]

2916
5731
% вакансий с требованиями:  50.9 %


In [298]:
print('len: ',vacs.shape[0])
display(vacs.head(2))

len:  2916


,id,description,specialization_id,flag,desc2
0,37003152,<p>Компания «Цифровое проектирование» ведет ис...,"[1.221, 1.25]",True,В зависимости от навыков и желаний Разработк...
1,37849676,"<p>Геоскан - российская группа компаний, заним...",[1.221],True,Опыт по сборке и подготовке дистрибутивов з...


In [299]:
vacs.iloc[100]['description']

'<p>Компания ПРИНТГРАД. Работаем с 2005 года.Занимаемся поддержкой ИТ-инфраструктуры. Ищем специалиста по монтажу и обслуживанию слаботочных систем:</p> <p><strong>Задачи:</strong></p> <ul> <li> <p>Монтаж СКС</p> </li> <li> <p>Первичная настройка компьютерного оборудования</p> </li> <li> <p>Обслуживание компьютерного оборудования</p> </li> </ul> <p><strong>Требования:</strong></p> <ul> <li> <p>Опыт монтажа СКС</p> </li> <li> <p>Первичная настройка компьютерного оборудования</p> </li> <li> <p>Обслуживание компьютерного оборудования</p> </li> </ul> <p><strong>Наше предложение:</strong></p> <ul> <li> <p>Доход от 35.000 руб./мес. Структура дохода обсуждается на собеседовании,зависит от производительности</p> </li> <li> <p>Официальное трудоустройство и белая заработная плата</p> </li> <li> <p>Оплата корпоративной сотовой связи</p> </li> <li> <p>Обучение работе с торговым оборудованием, системами видеонаблюдения, маршрутизаторами, серверами за счет компании</p> </li> <li> <p>Карьерный рост</

In [300]:
vacs.iloc[100]['desc2']

'   Опыт монтажа СКС   Первичная настройка компьютерного оборудования   Обслуживание компьютерного оборудования   Наше предложение   Доход от 35000 руб/мес Структура дохода обсуждается на собеседованиизависит от производительности   Официальное трудоустройство и белая заработная плата   Оплата корпоративной сотовой связи   Обучение работе с торговым оборудованием системами видеонаблюдения маршрутизаторами серверами за счет компании   Карьерный рост   График работы 5/2 с 900 до 1800      '